# **Analyse rapide de methadonnées**

L'objectif de ce notbook est de faire une breve analyse de methédonnée disponibles sur l'ensemble des images de radiographie de torse. 

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', append=True, category=UserWarning)
from aif360.datasets import StandardDataset
from aif360.algorithms.preprocessing import Reweighing
from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pd.read_csv('clem/metadata.csv')

# Analyse des Métadonnées et Prétraitement par Reweighting

Dans ce notebook, nous allons analyser les métadonnées d'un jeu de données médical où :
- Les **colonnes sensibles** sont `Patient Age` et `Patient Gender`.
- La **variable cible** est `Finding Labels`.  
  Si la valeur est `"No Finding"`, cela signifie qu'il n'y a **aucune maladie** détectée.  
  Dans le cas contraire, la chaîne contient une ou plusieurs maladies (séparées par `|`). Nousa allons donc simplement diviser les données en `malade` et `sain`

L'objectif est de :

1. Préparer les données en créant une variable binaire `disease` (0 pour *No Finding*, 1 pour présence de maladie).
2. Réaliser une analyse univariée et bivariée en utilisant Plotly Express pour visualiser les distributions et relations.
3. Calculer des poids pour chaque échantillon via un **reweighting** (pour compenser un éventuel déséquilibre de classes) et stocker ces poids dans la colonne `WEIGHTS` (déjà initialisés à 1.0).

In [ ]:
# Affichage d'un aperçu des données
print("Aperçu des premières lignes:")
display(df.head())

# Vérifier la présence des colonnes sensibles et de la colonne cible
print("Colonnes du DataFrame:", df.columns.tolist())

## 1. Préparation des données

In [ ]:
# Création de la variable cible binaire
#  - Si Finding Labels == "No Finding" => 0 (aucune maladie)
#  - Sinon => 1 (présence d'une ou plusieurs maladies)
df['disease'] = df['Finding Labels'].apply(lambda x: 0 if x == "No Finding" else 1)

In [ ]:
# Correction des âges incorrects : remplacement des âges > 140 par la médiane des âges valides
valid_age_mask = df['Patient Age'] <= 140
median_age = df.loc[valid_age_mask, 'Patient Age'].median()
df.loc[~valid_age_mask, 'Patient Age'] = median_age

In [ ]:
# Calcul de la distribution des malades/sains
distribution = df['disease'].value_counts().reset_index()
distribution.columns = ['disease', 'count']
distribution['disease'] = distribution['disease'].map({0: 'Sain', 1: 'Malade'})

# Affichage de la distribution avec un pie chart
fig_pie = px.pie(distribution, values='count', names='disease', 
                 title="Répartition des malades/sains parmi le jeu de données",
                 color_discrete_sequence=px.colors.qualitative.Pastel)
# fig_pie.show()

## 2. Analyse

### 2.1 Analyse Univariée

In [ ]:
# 1. Distribution de Patient Age
fig_age = px.histogram(df, 
                       x='Patient Age', 
                       title="Distribution de l'âge des patients",
                       nbins=40,
                       color_discrete_sequence=['skyblue'],
                       opacity=0.75,
                       labels={'Patient Age': 'Âge du patient'},
                       )
#fig_age.show()

In [ ]:
#2 Distribution du Genre

gender_distribution = df['Patient Gender'].value_counts().reset_index()
gender_distribution.columns = ['Patient Gender', 'count']

# Affichage de la distribution avec un pie chart
fig_pie_gender = px.pie(gender_distribution, values='count', names='Patient Gender',
                        title="Répartition du Genre ",
                        color_discrete_sequence=px.colors.qualitative.Pastel)
#fig_pie_gender.show()

### 2.2 Analyse bivariée

In [ ]:
# 1. Boxplot de Patient Age en fonction de la variable cible
fig_box_age = px.box(df, x='disease', y='Patient Age', 
                     title="Âge des patients selon la présence ou l'absence de maladie",
                     labels={'disease': 'Présence de maladie (0 = Sain, 1 = Malade)', 'Patient Age': 'Âge du patient'})
#fig_box_age.show()


In [ ]:
# Calcul du taux de malades par genre
df_gender_disease = df.groupby('Patient Gender')['disease'].value_counts(normalize=True).unstack().reset_index()

# Graphique à barres comparant le taux de malades chez les hommes et les femmes
fig_bar = px.bar(df_gender_disease, 
                 x='Patient Gender', 
                 y=[0, 1],  # 0 correspond aux sains, 1 correspond aux malades
                 labels={'Patient Gender': 'Genre', 'value': 'Taux de Malade'}, 
                 title="Comparaison du Taux de Malades entre Hommes et Femmes",
                 barmode='group',  
                 text_auto=True)  

#fig_bar.show()

On remarque qu'il y a une légère différence de répartition des non-malades entre femmes et hommes .

## Repondération

In [ ]:
dfr = df.copy()
label_encoder = LabelEncoder()
for column in dfr.select_dtypes(include=['object']).columns:
    dfr[column] = label_encoder.fit_transform(dfr[column])

Nous allons donc faire la ponderation de 3 façon differents. Le genre, l'âge, et la combinaison des 2, sachant que nous avons des biais visibles en fonction les tranches d'âge.

### Par le genre

In [ ]:
dataset_gender = StandardDataset(
    dfr,
    label_name="disease",              # Variable à prédire
    favorable_classes=[0],             # 0 = absence de maladie (classe favorable)
    protected_attribute_names=["Patient Gender"],
    privileged_classes=[[1]]           # Groupe privilégié : patients avec Patient Gender == 1
)
reweigher_gender = Reweighing(
    unprivileged_groups=[{'Patient Gender': 0}],  # Non privilégié
    privileged_groups=[{'Patient Gender': 1}]       # Privilégié
)
dataset_transf_gender = reweigher_gender.fit_transform(dataset_gender)
dfr['WEIGHTS_GENDER'] = dataset_transf_gender.instance_weights

### Par l'âge (selon les quartiles)

In [ ]:
# --- 3. Reweighting par l'âge ---
# On définit une variable binaire "Age Privileged" en fonction du label 'disease' et des intervalles d'âge :
#   - Pour les patients sans maladie (disease == 0) : 34 <= Patient Age <= 59 est privilégié
#   - Pour les patients malades (disease == 1) : 42 <= Patient Age <= 62 est privilégié
def age_privileged(row):
    if row['disease'] == 0:
        return int(34 <= row['Patient Age'] <= 59)
    else:
        return int(42 <= row['Patient Age'] <= 62)

dfr['Age Privileged'] = dfr.apply(age_privileged, axis=1)

dataset_age = StandardDataset(
    dfr,
    label_name="disease",              # Variable à prédire
    favorable_classes=[0],
    protected_attribute_names=["Age Privileged"],
    privileged_classes=[[1]]           # Groupe privilégié : âge dans l'intervalle défini
)
reweigher_age = Reweighing(
    unprivileged_groups=[{'Age Privileged': 0}],
    privileged_groups=[{'Age Privileged': 1}]
)
dataset_transf_age = reweigher_age.fit_transform(dataset_age)
dfr['WEIGHTS_AGE'] = dataset_transf_age.instance_weights

### Par les deux

In [ ]:
# On définit "Combined Privileged" comme 1 si le patient a à la fois Patient Gender == 1 et Age Privileged == 1, sinon 0
dfr['Combined Privileged'] = ((dfr['Patient Gender'] == 1) & (dfr['Age Privileged'] == 1)).astype(int)

dataset_combined = StandardDataset(
    dfr,
    label_name="disease",              # Variable à prédire
    favorable_classes=[0],
    protected_attribute_names=["Combined Privileged"],
    privileged_classes=[[1]]           # Groupe privilégié si les deux conditions sont remplies
)
reweigher_combined = Reweighing(
    unprivileged_groups=[{'Combined Privileged': 0}],
    privileged_groups=[{'Combined Privileged': 1}]
)
dataset_transf_combined = reweigher_combined.fit_transform(dataset_combined)
dfr['WEIGHTS_COMBINED'] = dataset_transf_combined.instance_weights

# --- Vérification rapide : affichage des 10 premières lignes avec les colonnes de poids ---
print(dfr[['Patient Gender', 'Patient Age', 'disease', 'Age Privileged', 'Combined Privileged',
           'WEIGHTS_GENDER', 'WEIGHTS_AGE', 'WEIGHTS_COMBINED']].head(10))

In [ ]:
df["WEIGHTS"] = dfr["WEIGHTS"]

In [ ]:
df0 = pd.read_csv('clem/metadata.csv')
df1 = df0.copy()
df2 = df1.copy()

In [ ]:
df0['WEIGHTS']=dfr['WEIGHTS_GENDER'].values

In [ ]:
df1['WEIGHTS']=dfr['WEIGHTS_AGE'].values

In [ ]:
df2['WEIGHTS']=dfr['WEIGHTS_COMBINED'].values

### Faire les csv pour le training

In [ ]:
df0.to_csv("./clem/metadata_gender.csv")
df1.to_csv("./clem/metadata_age.csv")
df2.to_csv("./clem/metadata_both.csv")

In [ ]:
df0.to_csv("./1_clem/metadata_gender.csv")
df1.to_csv("./1_clem/metadata_age.csv")
df2.to_csv("./1_clem/metadata_both.csv")